In [2]:
!pip install morph-kgc==2.6.4
import morph_kgc

  Using cached morph_kgc-2.6.4-py3-none-any.whl.metadata (10 kB)
  Using cached elementpath-4.4.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached jsonpath_python-1.0.6-py3-none-any.whl.metadata (12 kB)
  Using cached pyoxigraph-0.3.22-cp37-abi3-win_amd64.whl.metadata (5.3 kB)
  Using cached ruamel.yaml-0.18.0-py3-none-any.whl.metadata (19 kB)
  Using cached sql_metadata-2.11.0-py3-none-any.whl.metadata (9.8 kB)
  Using cached ruamel.yaml.clib-0.2.8-cp311-cp311-win_amd64.whl.metadata (2.3 kB)
  Using cached sqlparse-0.5.0-py3-none-any.whl.metadata (3.9 kB)
Using cached morph_kgc-2.6.4-py3-none-any.whl (53 kB)
Using cached ruamel.yaml-0.18.0-py3-none-any.whl (113 kB)
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   - -------------------------------------- 0.4/9.5 MB 12.5 MB/s eta 0:00:01
   --- ------------------------------------ 0.9/9.5 MB 9.0 MB/s eta 0:00:01
   ----- ---------------------------------- 1.3/9.5 MB 10.3 MB/s eta 0:00:01
   ------- --------------

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
config = """
             [GTFS-Madrid-Bench]
             mappings: https://raw.githubusercontent.com/oeg-upm/morph-kgc/main/examples/tutorial/mapping.gtfs.ttl
         """

In [4]:
g = morph_kgc.materialize(config)

INFO | 2024-05-22 10:13:46,351 | Parallelization is not supported for win32 when running as a library. If you need to speed up your data integration pipeline, please run through the command line.
c:\Users\tamjh\AppData\Local\Programs\Python\Python311\Lib\site-packages\morph_kgc\mapping\mapping_parser.py:390: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  fnml_df = fnml_df.applymap(str)
INFO | 2024-05-22 10:13:50,719 | 86 mapping rules retrieved.
c:\Users\tamjh\AppData\Local\Programs\Python\Python311\Lib\site-packages\morph_kgc\mapping\mapping_parser.py:485: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.rml_df = self.rml_df.replace(r'^\s*$', np.nan, regex=True)
INFO | 2024-05-22 10:13:50,843 | Mapping partition with 8

In [6]:
config = """
             [SoMEF]
             mappings: https://raw.githubusercontent.com/oeg-upm/morph-kgc/main/examples/tutorial/mapping.somef.ttl
         """

g = morph_kgc.materialize_oxigraph(config)

INFO | 2024-05-22 10:17:53,053 | Parallelization is not supported for win32 when running as a library. If you need to speed up your data integration pipeline, please run through the command line.
c:\Users\tamjh\AppData\Local\Programs\Python\Python311\Lib\site-packages\morph_kgc\mapping\mapping_parser.py:390: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  fnml_df = fnml_df.applymap(str)
INFO | 2024-05-22 10:17:57,944 | 235 mapping rules retrieved.
c:\Users\tamjh\AppData\Local\Programs\Python\Python311\Lib\site-packages\morph_kgc\mapping\mapping_parser.py:485: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.rml_df = self.rml_df.replace(r'^\s*$', np.nan, regex=True)
INFO | 2024-05-22 10:17:58,145 | Mapping partition with 

In [7]:
q = """
         PREFIX sd: <https://w3id.org/okn/o/sd#>
         PREFIX em: <https://www.w3id.org/okn/o/em#>

         SELECT * WHERE {
             ?sowtware a sd:Software .
             << ?software sd:license ?license >> em:confidence ?confidence .
             << ?software sd:license ?license >> em:technique ?technique .
         }
    """

q_res = g.query(q)

for solution in q_res:
    print(solution['software'], solution ['license'], solution ['technique'], solution['confidence'])

<https://www.w3id.org/okn/i/Software/oeg-upm/morph-kgc> "https://api.github.com/licenses/apache-2.0"^^<http://www.w3.org/2001/XMLSchema#anyURI> "GitHub API" "1.0"


In [ ]:
import os

cwd = os.path.dirname(os.getcwd())

config = f"""
[StudentSportDB]
mappings={cwd}/dat/student_sport.r2rml.ttl
db_url=sqlite:///{cwd}/dat/student_sport.db
"""

print(config)

This config references a R2RML mapping (in student_sport.r2rml.ttl) which gets applied to the input data. One way to create such a mapping is YARRRML, "a human-friendly text-based representation of RML rules". You write rules in YAML then transform these with a yarrrml-parser into either a RML or R2RML file. See the browser-based version called Matey to get an idea of how this works.

The YARRRML file needed for this example can be found in dat/ which can be used with the sample database:

prefixes:
  ns1: "http://example.com/"

sources:
  students:
    queryFormulation: mysql
    query: select * from Student left join Student_Sport on Student.ID = Student_Sport.ID_Student left join Sport on Sport.ID = Student_Sport.ID_Sport;

mappings:
  student:
    sources: students
    s: http://example.com/$(ID_Student)
    po:
      - [a, ns1:Student]
      - [ns1:firstName, $(FirstName)]
      - [ns1:lastName, $(LastName)]
      - p: ns1:plays
        o: 
        - mapping: sport
          condition:
            function: equal
            parameters:
              - [str1, $(ID_Sport)]
              - [str2, $(ID_Sport)]
  sport:
    sources: students
    s: http://example.com/$(ID_Sport)
    po:
      - [a, ns1:Sport]
      - [ns1:description, $(Description)]
      - [ns1:id, $(ID_Sport)]
Using the yarrrml-parser:

yarrrml-parser -i student_sport.yml -o student_sport.r2rml.ttl -f R2RML
produces the students_sport.r2rml.ttl file. This is the file referenced in the configuration for Morph-KGC.